In [1]:
%matplotlib inline

import pandas as pd
import numpy as np

columns = [
    'id', 'testing_for', 'address', 'zip_code', 'provider',
    'connected_with', 'monthly_price', 'provider_down_speed',
    'provider_price', 'actual_down_speed', 'actual_price', 'rating',
    'completed', 'created_at', 'updated_at', 'latitude', 'longitude',
    'ping', 'actual_upload_speed', 'test_id', 'ip_address', 'hostname',
    'from_mlab', 'area_code', 'test_type', 'census_code',
    'upload_median', 'download_median', 'census_status', 'test_date',
    'country_code', 'region', 'county', 'accuracy', 'location',
    'census_block'
]

# Read CSV in to Dataframe
filename = './data/3cb026d0-0ad8-45c3-a030-cd5b19c2ab17.csv'
submissions = pd.read_csv(filename, header=None, names=columns)

submissions.tail()

,id,testing_for,address,zip_code,provider,connected_with,monthly_price,provider_down_speed,provider_price,actual_down_speed,...,upload_median,download_median,census_status,test_date,country_code,region,county,accuracy,location,census_block
1470,8684792,Home Wifi,NaN,97493.0,LS Networks,NaN,40.0,14.0,2.86,4.14,...,NaN,NaN,NaN,2019-11-28,US,OR,41039.0,50.0,NaN,4.103900e+14
1471,8684793,Home Wifi,NaN,97401.0,"Comcast Cable Communications, LLC","Wireless connection, multiple devices in house...",NaN,NaN,NaN,93.40,...,NaN,NaN,NaN,2019-11-29,US,OR,41039.0,5018.0,NaN,4.103900e+14
1472,8684794,Home Wifi,NaN,97426.0,"CenturyLink Communications, LLC",NaN,27.0,0.0,NaN,0.24,...,NaN,NaN,NaN,2019-11-29,US,OR,41039.0,50.0,NaN,4.103900e+14
1473,8684795,Home Wifi,NaN,97471.0,Douglas FastNet,NaN,50.0,100.0,0.50,91.62,...,NaN,NaN,NaN,2019-11-29,US,OR,41019.0,50.0,NaN,4.101907e+14
1474,8684796,Home Wifi,NaN,97471.0,Douglas FastNet,NaN,50.0,50.0,1.00,286.21,...,NaN,NaN,NaN,2019-11-29,US,OR,41019.0,50.0,NaN,4.101915e+14


In [2]:
# Cleanup
submissions['rating'] = submissions['rating'].replace(0.0,  np.nan)
submissions['census_code'].dropna(inplace=True)
submissions['census_block'].dropna(inplace=True)

In [3]:
# Prepare Lane County DataFrame
lane = submissions[submissions['county'] == 41039].copy()
lane = lane[lane['test_type'] != 'duplicate']
lane = lane[lane['census_code'].notnull()]
lane = lane[lane['census_block'].notnull()]

# Convert census_codes to integer from float64
lane['census_code'] = lane['census_code'].astype(np.int64)
lane['census_block'] = lane['census_block'].astype(np.int64)

In [4]:
lane.describe()

,zip_code,monthly_price,provider_down_speed,provider_price,actual_down_speed,actual_price,rating,completed,latitude,longitude,...,actual_upload_speed,from_mlab,area_code,census_code,upload_median,download_median,county,accuracy,location,census_block
count,798.000000,465.000000,381.000000,358.000000,799.000000,358.000000,493.000000,799.000000,799.000000,799.000000,...,799.000000,799.0,0.0,7.990000e+02,0.0,0.0,799.0,327.000000,0.0,7.990000e+02
mean,97423.340852,76.578495,114.805774,5.320140,54.049975,31.513659,4.052738,0.984981,44.041495,-123.122905,...,20.789287,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,2419.816514,NaN,4.103900e+14
std,30.292992,41.102725,191.766289,12.343979,76.652456,169.584727,1.829428,0.121704,0.078003,0.247992,...,73.479865,0.0,NaN,1.557435e+03,NaN,NaN,0.0,13968.923479,NaN,1.557425e+07
min,97401.000000,0.000000,0.000000,0.010000,0.050000,0.040000,1.000000,0.000000,43.686800,-124.130500,...,0.000000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,6.000000,NaN,4.103900e+14
25%,97402.000000,50.000000,15.000000,0.600000,7.325000,1.170000,3.000000,1.000000,44.027900,-123.146750,...,1.440000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,50.000000,NaN,4.103900e+14
50%,97405.000000,69.000000,60.000000,1.305000,24.420000,3.035000,4.000000,1.000000,44.050000,-123.094000,...,5.720000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,54.000000,NaN,4.103900e+14
75%,97439.000000,90.000000,120.000000,4.280000,68.855000,10.972500,5.000000,1.000000,44.075200,-123.057400,...,11.350000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,1062.000000,NaN,4.103900e+14
max,97493.000000,200.000000,1500.000000,120.000000,591.840000,2400.000000,7.000000,1.000000,44.262200,-122.154200,...,864.820000,0.0,NaN,4.103901e+10,NaN,NaN,41039.0,154677.000000,NaN,4.103901e+14


In [5]:
lane_tracts = lane['census_code'].unique()
len(lane_tracts)

86

## Census Tract assignments

In [6]:
# Hand built by clicking around on speedupamerica.com's results page
# Next time I will use https://tigerweb.geo.census.gov/tigerweb/
eugene_tracts = [
    41039002201, 41039002202, 41039002301, 41039002401,
    41039002302, 41039002403, 41039002404, 41039002501,
    41039002503, 41039002504, 41039002600, 41039002700,
    41039002800, 41039002902, 41039002903, 41039002904,
    41039003000, 41039003101, 41039003102, 41039003600,
    41039003700, 41039003800, 41039003900, 41039004000,
    41039004100, 41039004200, 41039004300, 41039004401,
    41039004403, 41039004404, 41039004405, 41039004501,
    41039004502, 41039004600, 41039004700, 41039004800,
    41039004900, 41039005000, 41039005100, 41039005300,
    41039005400
]
springfield_tracts = [
    41039001801, 41039001803, 41039001804, # Thurston
    41039001902, 41039001903, 41039001904, # Centeral Springfield
    41039002001, 41039002002, # North Springfield
    41039002101, 41039002102, # Gateway
    41039003201, 41039003202, # West Springfield (Centenial)
    41039003301, 41039003302, # Springfield
    41039003400, # East of Mohawk
    41039003500 # Jasper Rd and Dorris Ranch
]

## Eugene

In [7]:
eugene = lane[lane['census_code'].isin(eugene_tracts)]
eugene.describe()

,zip_code,monthly_price,provider_down_speed,provider_price,actual_down_speed,actual_price,rating,completed,latitude,longitude,...,actual_upload_speed,from_mlab,area_code,census_code,upload_median,download_median,county,accuracy,location,census_block
count,452.000000,226.000000,197.000000,181.000000,452.000000,181.000000,267.000000,452.000000,452.000000,452.000000,...,452.000000,452.0,0.0,4.520000e+02,0.0,0.0,452.0,198.000000,0.0,4.520000e+02
mean,97402.931416,75.119469,159.284264,3.315525,71.862190,6.530110,4.632959,0.975664,44.057154,-123.106555,...,31.064867,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,2499.257576,NaN,4.103900e+14
std,1.972580,38.997879,221.541401,10.499106,90.054899,19.411991,1.629186,0.154262,0.027071,0.032842,...,95.100435,0.0,NaN,9.225587e+02,NaN,NaN,0.0,14351.966839,NaN,9.225693e+06
min,97401.000000,0.000000,0.000000,0.010000,0.120000,0.040000,1.000000,0.000000,43.993300,-123.200500,...,0.100000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,6.000000,NaN,4.103900e+14
25%,97401.000000,50.000000,45.000000,0.500000,13.905000,0.870000,4.000000,1.000000,44.040125,-123.128025,...,4.317500,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,48.000000,NaN,4.103900e+14
50%,97402.000000,70.000000,80.000000,1.000000,40.715000,1.730000,5.000000,1.000000,44.050550,-123.097100,...,5.965000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,50.000000,NaN,4.103900e+14
75%,97405.000000,89.750000,200.000000,1.830000,86.980000,4.600000,6.000000,1.000000,44.077550,-123.086300,...,11.780000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,1082.250000,NaN,4.103900e+14
max,97408.000000,200.000000,1000.000000,120.000000,591.840000,200.000000,7.000000,1.000000,44.131400,-123.024100,...,864.820000,0.0,NaN,4.103901e+10,NaN,NaN,41039.0,154677.000000,NaN,4.103901e+14


In [8]:
eugene.groupby('provider').agg({
    "id": ["count"],
    "rating":  ["mean", "median", "count"],
    "actual_down_speed": ["mean", "median", "count"],
    "monthly_price": ["mean", "median", "count"],
    "actual_price": ["mean", "median", "count"]
}).sort_values(('id', 'count'), ascending=False).head(15)

id    rating         \
                                                   count      mean median   
provider                                                                    
Comcast Cable Communications, LLC                    285  4.421687    4.0   
XS Media                                              59  5.744681    6.0   
CenturyLink Communications, LLC                       57  3.636364    4.0   
Network for Education and Research in Oregon (N...    14  5.800000    6.0   
Verizon Wireless                                       9  4.000000    4.0   
AT&T Mobility LLC                                      5  4.000000    4.0   
Emerald Broadband, LLC                                 5  5.400000    7.0   
University of Oregon                                   4  7.000000    7.0   
Sprint Personal Communications Systems                 3  3.000000    3.0   
ATT                                                    2       NaN    NaN   
Douglas FastNet                                        2  7.000000    7.0   
Charter Communications                                 1       NaN    NaN   
Cogent Communications                                  1  7.000000    7.0   
Peak Internet, LLC                                     1       NaN    NaN   
Sprint                                                 1       NaN    NaN   

                                                         actual_down_speed  \
                                                   count              mean   
provider                                                                     
Comcast Cable Communications, LLC                    166         79.963649   
XS Media                                              47        111.116949   
CenturyLink Communications, LLC                       33         16.436842   
Network for Education and Research in Oregon (N...     5         51.356429   
Verizon Wireless                                       3         12.237778   
AT&T Mobility LLC                                      2         11.554000   
Emerald Broadband, LLC                                 5         29.784000   
University of Oregon                                   1         44.880000   
Sprint Personal Communications Systems                 1         10.126667   
ATT                                                    0         86.675000   
Douglas FastNet                                        2        145.640000   
Charter Communications                                 0         31.800000   
Cogent Communications                                  1        265.150000   
Peak Internet, LLC                                     0          7.630000   
Sprint                                                 0         16.220000   

                                                                   \
                                                     median count   
provider                                                            
Comcast Cable Communications, LLC                    58.650   285   
XS Media                                             60.780    59   
CenturyLink Communications, LLC                       7.720    57   
Network for Education and Research in Oregon (N...   26.655    14   
Verizon Wireless                                      6.460     9   
AT&T Mobility LLC                                     9.620     5   
Emerald Broadband, LLC                               28.820     5   
University of Oregon                                 44.885     4   
Sprint Personal Communications Systems                1.470     3   
ATT                                                  86.675     2   
Douglas FastNet                                     145.640     2   
Charter Communications                               31.800     1   
Cogent Communications                               265.150     1   
Peak Internet, LLC                                    7.630     1   
Sprint                                               16.220     1   

                   

In [9]:
eugene.groupby('census_code').agg({
    "id": ["count"],
    "rating":  ["mean", "median", "count"],
    "actual_down_speed": ["mean", "median", "count"],
    "monthly_price": ["mean", "median", "count"],
    "actual_price": ["mean", "median", "count"]
})

id    rating              actual_down_speed                 \
            count      mean median count              mean   median count   
census_code                                                                 
41039002201    10  5.000000    5.0     6         17.754000   11.155    10   
41039002202    13  4.166667    4.0     6         73.026923   51.560    13   
41039002301     2  3.500000    3.5     2         81.890000   81.890     2   
41039002302    10  5.000000    5.0     6         66.483000   35.645    10   
41039002401     4  5.000000    5.0     4         54.692500   45.430     4   
41039002403     4  4.000000    4.0     2         70.337500   49.365     4   
41039002404     7  3.500000    3.5     4         40.652857   33.950     7   
41039002501     2  4.000000    4.0     1        159.070000  159.070     2   
41039002503    16  4.142857    4.0     7        141.129375   56.340    16   
41039002504     4       NaN    NaN     0         91.875000   72.285     4   
41039002600     7  3.333333    3.0     3         17.928571   10.080     7   
41039002700    11  4.666667    5.0     9         45.190909   21.360    11   
41039002800    10  5.571429    7.0     7         39.703000   25.115    10   
41039002902     8  4.400000    4.0     5         97.241250   88.775     8   
41039002903     7  3.500000    3.5     4         54.781429   66.220     7   
41039002904     7  4.250000    4.0     4         34.954286   35.960     7   
41039003000    16  6.000000    6.0     8         60.376250   51.005    16   
41039003101    10  3.833333    4.0     6         39.456000   29.655    10   
41039003102    14  3.125000    3.0     8         37.556429   23.475    14   
41039003600     7  3.800000    4.0     5         26.350000    8.860     7   
41039003700     8  5.000000    4.5     4         65.412500   47.895     8   
41039003800    12  5.222222    5.0     9         65.299167   45.250    12   
41039003900    43  5.304348    5.0    23         69.118837   54.290    43   
41039004000    28  5.458333    6.0    24         58.864643   30.360    28   
41039004100     2  1.000000    1.0     1        100.890000  100.890     2   
41039004200     7  4.250000    4.5     4         58.124286   58.080     7   
41039004300     9  4.333333    5.0     3         60.977778   11.020     9   
41039004401    15  5.000000    5.0    12        136.168000   67.850    15   
41039004403     9  4.666667    4.5     6         72.590000   67.930     9   
41039004404     5  3.250000    3.5     4         85.900000   69.090     5   
41039004405    14  5.166667    5.5     6         78.442143   45.285    14   
41039004501     6  2.250000    2.0     4         22.563333   14.720     6   
41039004502    44  5.130435    6.0    23        111.449545   66.820    44   
41039004600     6       NaN    NaN     0         50.621667   30.730     6   
41039004700    28  4.300000    5.0    20         47.212143   45.185    28   
41039004800     6  3.500000    4.0     6         34.390000   22.090     6   
41039004900     4  5.000000    5.0     3         76.147500   69.280     4   
41039005000    11  3.142857    4.0     7         80.762727   36.770    11   
41039005100     5  6.000000    6.0     2        159.996000  137.890     5   
41039005300     6  5.500000    5.5     2        166.590000  178.065     6   
41039005400    15  4.857143    5.0     7         75.295333   43.260    15   

            monthly_price              actual_price                 
                     mean median count         mean   median count  
census_code                                                         
41039002201     85.000000   85.0     2     1.270000    1.270     1  
41039002202     99.166667   72.5     6     7.315000    1.345     4  
41039002301     50.000000   50.0     1          NaN      NaN     0  
41039002302     68.000000   75.0     5     1.734000    1.480     5  
41039002401     76.000000   80.0     3     0.660000    0.660     1  
41039002403    103.333333  100.0     3     1.215000    1.215     2  
41039

## Springfield

In [10]:
springfield = lane[lane['census_code'].isin(springfield_tracts)]
springfield.describe()

,zip_code,monthly_price,provider_down_speed,provider_price,actual_down_speed,actual_price,rating,completed,latitude,longitude,...,actual_upload_speed,from_mlab,area_code,census_code,upload_median,download_median,county,accuracy,location,census_block
count,86.000000,59.000000,50.000000,50.000000,86.000000,50.000000,60.000000,86.000000,86.000000,86.000000,...,86.000000,86.0,0.0,8.600000e+01,0.0,0.0,86.0,47.000000,0.0,8.600000e+01
mean,97477.360465,75.525424,105.320000,3.222000,54.487093,8.235400,3.966667,0.988372,44.055135,-122.985176,...,7.303953,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,1634.787234,NaN,4.103900e+14
std,0.482951,39.144893,125.621791,4.972896,59.804159,19.683492,1.794215,0.107833,0.013657,0.045834,...,7.288785,0.0,NaN,6.619374e+02,NaN,NaN,0.0,5360.827410,NaN,6.619557e+06
min,97477.000000,0.000000,3.000000,0.030000,0.110000,0.520000,1.000000,0.000000,44.011700,-123.043000,...,0.260000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,11.000000,NaN,4.103900e+14
25%,97477.000000,47.500000,21.000000,0.607500,13.395000,0.807500,2.750000,1.000000,44.046200,-123.022000,...,3.462500,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,50.000000,NaN,4.103900e+14
50%,97477.000000,70.000000,60.000000,0.890000,32.715000,2.040000,4.000000,1.000000,44.052950,-122.999900,...,5.740000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,54.000000,NaN,4.103900e+14
75%,97478.000000,93.500000,142.500000,3.987500,72.925000,7.117500,5.000000,1.000000,44.067925,-122.936600,...,11.040000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,781.000000,NaN,4.103900e+14
max,97478.000000,180.000000,500.000000,26.670000,311.280000,125.000000,7.000000,1.000000,44.078400,-122.874300,...,40.510000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,33543.000000,NaN,4.103900e+14


In [11]:
springfield.groupby('provider').agg({
    "id": ["count"],
    "rating":  ["mean", "median", "count"],
    "actual_down_speed": ["mean", "median", "count"],
    "monthly_price": ["mean", "median", "count"],
    "actual_price": ["mean", "median", "count"]
}).sort_values(('id', 'count'), ascending=False).head(15)

id    rating         \
                                                   count      mean median   
provider                                                                    
Comcast Cable Communications, LLC                     63  4.195652    4.0   
CenturyLink Communications, LLC                       16  3.700000    4.5   
Verizon Wireless                                       2       NaN    NaN   
AVAST Software s.r.o.                                  1  2.000000    2.0   
Network for Education and Research in Oregon (N...     1       NaN    NaN   
SoftLayer Technologies Inc.                            1  1.000000    1.0   
Sprint                                                 1  3.000000    3.0   
ViaSat,Inc.                                            1  2.000000    2.0   

                                                         actual_down_speed  \
                                                   count              mean   
provider                                                                     
Comcast Cable Communications, LLC                     46         69.537937   
CenturyLink Communications, LLC                       10         10.206875   
Verizon Wireless                                       0          1.785000   
AVAST Software s.r.o.                                  1         31.720000   
Network for Education and Research in Oregon (N...     0          2.730000   
SoftLayer Technologies Inc.                            1         23.900000   
Sprint                                                 1         76.380000   
ViaSat,Inc.                                            1          3.390000   

                                                                  \
                                                    median count   
provider                                                           
Comcast Cable Communications, LLC                   53.990    63   
CenturyLink Communications, LLC                      8.835    16   
Verizon Wireless                                     1.785     2   
AVAST Software s.r.o.                               31.720     1   
Network for Education and Research in Oregon (N...   2.730     1   
SoftLayer Technologies Inc.                         23.900     1   
Sprint                                              76.380     1   
ViaSat,Inc.                                          3.390     1   

                                                   monthly_price               \
                                                            mean median count   
provider                                                                        
Comcast Cable Communications, LLC                      68.904762   68.5    42   
CenturyLink Communications, LLC                        97.785714   89.5    14   
Verizon Wireless                                             NaN    NaN     0   
AVAST Software s.r.o.                                  60.000000   60.0     1   
Network for Education and Research in Oregon (N...           NaN    NaN     0   
SoftLayer Technologies Inc.                            13.000000   13.0     1   
Sprint                                                       NaN    NaN     0   
ViaSat,Inc.                                           120.000000  120.0     1   

                                                   actual_price               
                                                           mean median count  
provider                                                                      
Comcast Cable Communications, LLC                      2.232432   1.33    37  
CenturyLink Communications, LLC                       26.657273  13.40    11  
Verizon Wireless                                            NaN    NaN     0  
AVAST Software s.r.o.                                       NaN    NaN     0  
Network for Education and Research in Oregon (N...          NaN    NaN     0  
SoftLayer Technologies Inc.                            0.540000   0.54    

In [12]:
springfield.groupby('census_code').agg({
    "id": ["count"],
    "rating":  ["mean", "median", "count"],
    "actual_down_speed": ["mean", "median", "count"],
    "monthly_price": ["mean", "median", "count"],
    "actual_price": ["mean", "median", "count"]
}).sort_values(('id', 'count'), ascending=False).head(15)

id    rating              actual_down_speed                 \
            count      mean median count              mean   median count   
census_code                                                                 
41039002002    13  3.666667    3.0    12         46.122308   49.860    13   
41039003302     9  3.428571    3.0     7         38.030000   26.580     9   
41039001801     7  5.000000    5.5     6         31.592857    8.380     7   
41039002102     7  5.250000    5.5     4         66.508571   46.210     7   
41039001803     6  4.250000    4.5     4         64.750000   56.845     6   
41039001804     6  2.666667    3.0     3         50.840000   15.725     6   
41039001902     5  6.250000    6.5     4         59.430000   31.410     5   
41039002001     5  3.250000    3.0     4         51.646000   36.930     5   
41039003202     5  3.000000    3.0     5        152.264000   89.370     5   
41039003500     5  3.500000    3.5     2         42.188000   29.540     5   
41039002101     4  4.500000    4.5     2        103.632500  109.575     4   
41039003201     4  3.000000    3.0     1         28.467500   28.270     4   
41039003301     4  3.000000    3.0     2         31.392500   20.975     4   
41039001903     2  3.500000    3.5     2         40.860000   40.860     2   
41039001904     2  5.000000    5.0     1         13.145000   13.145     2   

            monthly_price              actual_price               
                     mean median count         mean median count  
census_code                                                       
41039002002     53.600000   45.0    10     1.453333   0.66     9  
41039003302     85.666667   87.0     6    13.014000   3.20     5  
41039001801    117.600000  136.0     5    36.480000  13.40     5  
41039002102     64.400000   73.0     5     3.800000   2.21     5  
41039001803     64.000000   90.0     3     4.450000   4.45     2  
41039001804     77.400000   80.0     5    15.637500   2.89     4  
41039001902     74.250000   65.0     4     1.810000   2.31     3  
41039002001     57.250000   64.5     4     2.150000   2.15     1  
41039003202    114.500000  135.0     4     0.772500   0.55     4  
41039003500    115.000000   95.0     3     8.953333  12.63     3  
41039002101     70.000000   70.0     2     4.340000   4.34     1  
41039003201     41.666667   50.0     3     1.476667   1.49     3  
41039003301     65.000000   65.0     1     7.550000   7.55     1  
41039001903     72.500000   72.5     2     2.000000   2.00     2  
41039001904     45.000000   45.0     1     1.970000   1.97     1

## Lane County

In [13]:
county_tracts = np.setdiff1d(lane_tracts, eugene_tracts)
county_tracts = np.setdiff1d(county_tracts, springfield_tracts)

county = lane[lane['census_code'].isin(county_tracts)]
county.describe()

,zip_code,monthly_price,provider_down_speed,provider_price,actual_down_speed,actual_price,rating,completed,latitude,longitude,...,actual_upload_speed,from_mlab,area_code,census_code,upload_median,download_median,county,accuracy,location,census_block
count,260.000000,180.000000,134.000000,127.000000,261.000000,127.000000,166.000000,261.0,261.000000,261.000000,...,261.000000,261.0,0.0,2.610000e+02,0.0,0.0,261.0,82.000000,0.0,2.610000e+02
mean,97440.953846,78.755556,52.955224,9.003150,23.058736,76.284803,3.150602,1.0,44.009882,-123.196601,...,7.437471,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,2677.951220,NaN,4.103900e+14
std,28.455170,44.309428,141.801877,15.602687,35.791074,278.680377,1.784164,0.0,0.125903,0.417143,...,21.062046,0.0,NaN,7.331812e+02,NaN,NaN,0.0,16359.067743,NaN,7.331619e+06
min,97402.000000,20.000000,0.000000,0.110000,0.050000,0.350000,1.000000,1.0,43.686800,-124.130500,...,0.000000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,10.000000,NaN,4.103900e+14
25%,97424.000000,45.000000,10.000000,1.040000,2.670000,3.870000,1.000000,1.0,43.936300,-123.350600,...,0.620000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,50.000000,NaN,4.103900e+14
50%,97438.000000,65.000000,17.500000,3.750000,8.700000,10.770000,3.000000,1.0,44.023400,-123.178000,...,2.010000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,56.000000,NaN,4.103900e+14
75%,97463.000000,98.000000,60.000000,8.300000,24.790000,26.190000,4.000000,1.0,44.085600,-123.016000,...,8.050000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,165.000000,NaN,4.103900e+14
max,97493.000000,200.000000,1500.000000,90.000000,267.950000,2400.000000,7.000000,1.0,44.262200,-122.154200,...,209.590000,0.0,NaN,4.103901e+10,NaN,NaN,41039.0,147306.000000,NaN,4.103901e+14


In [14]:
county.groupby('provider').agg({
    "id": ["count"],
    "rating":  ["mean", "median", "count"],
    "actual_down_speed": ["mean", "median", "count"],
    "monthly_price": ["mean", "median", "count"],
    "actual_price": ["mean", "median", "count"]
}).sort_values(('id', 'count'), ascending=False).head(15)

id    rating               \
                                       count      mean median count   
provider                                                              
CenturyLink Communications, LLC          102  2.549296    2.0    71   
Charter Communications                    59  4.424242    5.0    33   
Verizon Wireless                          21  2.272727    2.0    11   
ViaSat,Inc.                               19  1.923077    2.0    13   
Comcast Cable Communications, LLC         14  4.500000    4.5     8   
XS Media                                  10  4.222222    4.0     9   
Hughes Network Systems                     9  2.400000    3.0     5   
Emerald Broadband, LLC                     8  3.142857    3.0     7   
Hunter Communications                      7  2.750000    3.0     4   
LS Networks                                5  5.000000    5.0     4   
AT&T Mobility LLC                          3       NaN    NaN     0   
Monroe Telephone Co.                       1  7.000000    7.0     1   
Sprint                                     1       NaN    NaN     0   
Sprint Personal Communications Systems     1       NaN    NaN     0   
tfm associates                             1       NaN    NaN     0   

                                       actual_down_speed                 \
                                                    mean   median count   
provider                                                                  
CenturyLink Communications, LLC                 6.356569    3.890   102   
Charter Communications                         48.948644   34.500    59   
Verizon Wireless                               18.085238    7.730    21   
ViaSat,Inc.                                     7.131053    5.860    19   
Comcast Cable Communications, LLC              59.479286   49.500    14   
XS Media                                       23.408000   19.790    10   
Hughes Network Systems                          5.138889    2.100     9   
Emerald Broadband, LLC                         18.511250    6.115     8   
Hunter Communications                          53.195714    7.160     7   
LS Networks                                    19.260000   18.700     5   
AT&T Mobility LLC                              12.403333    6.710     3   
Monroe Telephone Co.                            9.820000    9.820     1   
Sprint                                        115.560000  115.560     1   
Sprint Personal Communications Systems          2.920000    2.920     1   
tfm associates                                 71.400000   71.400     1   

                                       monthly_price               \
                                                mean median count   
provider                                                            
CenturyLink Communications, LLC            62.906667   55.0    75   
Charter Communications                     68.285714   65.0    35   
Verizon Wireless                          117.333333  107.5    12   
ViaSat,Inc.                               123.894737  120.0    19   
Comcast Cable Communications, LLC          70.000000   61.5    10   
XS Media                                  118.800000  119.0     5   
Hughes Network Systems                     90.000000   92.5     8   
Emerald Broadband, LLC                    114.285714   50.0     7   
Hunter Communications                      71.500000   62.5     4   
LS Networks                                40.250000   40.0     4   
AT&T Mobility LLC                                NaN    NaN     0   
Monroe Telephone Co.                       45.000000   45.0     1   
Sprint                                           NaN    NaN     0   
Sprint Personal Communications Systems           NaN    NaN     0   
tfm associates                                   NaN    NaN     0   

                                       actual_price                 
                                               mean   median count  
provider                       

In [15]:
county.groupby('census_code').agg({
    "id": ["count"],
    "rating":  ["mean", "median", "count"],
    "actual_down_speed": ["mean", "median", "count"],
    "monthly_price": ["mean", "median", "count"],
    "actual_price": ["mean", "median", "count"]
}).sort_values(('id', 'count'), ascending=False).head(15)

id    rating              actual_down_speed                \
            count      mean median count              mean  median count   
census_code                                                                
41039000100    23  2.937500    3.0    16          9.584783   6.650    23   
41039001002    23  2.187500    1.5    16          9.400870   5.700    23   
41039000902    18  4.384615    5.0    13         18.768889   5.795    18   
41039000903    17  3.727273    3.0    11         11.372941   3.120    17   
41039000200    15  2.000000    2.0    11         12.948000   5.820    15   
41039001101    15  3.333333    3.5     6         45.209333  10.340    15   
41039000904    13  2.750000    2.0     8         22.722308  10.080    13   
41039000403    11  4.600000    4.0     5         32.150000   9.820    11   
41039001102    10  3.000000    4.0     5         47.752000  21.050    10   
41039000706    10  3.400000    4.0     5         44.047000  29.600    10   
41039000800    10  2.555556    3.0     9          5.025000   0.800    10   
41039000402    10  2.666667    3.0     9          6.893000   3.490    10   
41039001201     9  3.000000    3.0     5         24.513333   5.410     9   
41039001700     8  2.500000    2.5     2         35.956250  12.020     8   
41039001500     8  3.500000    2.5     6         20.543750   6.050     8   

            monthly_price              actual_price                 
                     mean median count         mean   median count  
census_code                                                         
41039000100     51.944444   48.5    18    12.191333   15.380    15  
41039001002    117.684211  105.0    19   162.656000   30.615    10  
41039000902     87.142857   85.0    14   106.293000   19.610    10  
41039000903     60.181818   58.0    11    79.232857   10.630     7  
41039000200     95.000000   85.0    10    30.533750   11.315     8  
41039001101     89.833333   62.5     6    19.386667   16.670     3  
41039000904     82.100000   72.5    10     7.673750    5.960     8  
41039000403     57.600000   50.0     5    14.286667   16.390     3  
41039001102     63.166667   61.0     6    32.050000   27.140     4  
41039000706     80.200000   60.0     5     4.022000    1.970     5  
41039000800     89.444444   90.0     9   224.890000  145.255     4  
41039000402     91.600000   77.5    10   198.492500   18.415     8  
41039001201     76.000000   75.5     6     6.506667    7.940     3  
41039001700     80.400000   70.0     5     7.570000    7.055     4  
41039001500     69.800000   65.0     5    18.662000   10.890     5